# Link Prediction in Condmat

In [29]:
import collections
import copy
import datetime
import itertools
import math
from typing import List, Any, Dict, Tuple

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV, train_test_split
import seaborn as sns
from tqdm import tqdm
from xgboost import XGBClassifier

# Typing
NodePair = Tuple[int, int]
Edge = List[Tuple[int, int, Dict['date', datetime.datetime]]]

folder = '/local/bruingjde/complexnetworks2020-experiment/temp/en-a1'

In [30]:
def read_edges(file='out.enron') -> pd.DataFrame:
  d = pd.read_csv('out.enron', sep=' ', skiprows=1, names=['source', 'target', 'weight', 'date'])
  d['date'] = d['date'].apply(datetime.datetime.fromtimestamp)
  d.sort_values(by='date', inplace=True)
  return d.loc[:, ['source', 'target', 'date']]
def filter_edgelist(edges: pd.DataFrame, start=0, stop=1, verbose=True) -> pd.DataFrame: 
  """Filter edgelist.  If start/ stop is float, start/stop from the fraction of total edges. If datetime, this is used.""" 
  no_edges = len(edges)
  if start != 0:
    if type(start) is float:
      assert 0 < start < 1
      start = int(start*no_edges)
    if type(start) is int: start = edges.iloc[start]['date']
    start = start + datetime.timedelta(seconds=1)
  else: start = edges['date'].min()
  if verbose: print(start)
  
  if stop != 1:
    if type(stop) is float:
      assert 0 < stop < 1
      stop = math.floor(stop*no_edges)-1
    if type(stop) is int: stop = edges.iloc[stop]['date']
  else: stop = edges['date'].max()
  if verbose: print(stop)
  
  mask = (edges['date'] >= start) & (edges['date'] <= stop)
  if verbose: 
    no_selected_edges = sum(mask)
    print(f'{no_selected_edges=} ({no_selected_edges/len(edges):.1e})')

  return edges.loc[mask]
def convert_to_set(edges: pd.DataFrame) -> List[NodePair]: return {edge for edge in edges.loc[:, ['source', 'target']].itertuples(index=False, name=None)}
def get_graph(edgelist: pd.DataFrame) -> nx.Graph:
  """Add edge to graph. Contains edge attribute weight."""
  g = nx.Graph()
  
  for u, v, _ in edgelist.itertuples(index=False, name=None):
    weight = g[u][v]["weight"]+1 if g.has_edge(u,v) else 1
    g.add_edge(u, v, weight=weight)
  
  return g
def giant_component(graph: nx.Graph) -> nx.Graph: return graph.subgraph(max(nx.connected_components(graph), key=len)).copy()
def report(graph:nx.Graph, probes: Tuple[int, int]):
  n = len(probes)
  print(f"Number of probes: {n}")
  a = sum([graph.has_edge(u, v) for u, v in probes])
  print(f"- already edge: {a} ({a/n:.0%})")
  non_edges = set(nx.non_edges(graph))
  ne = sum([np in non_edges for np in probes])
  print(f"- both nodes in graph: {ne} ({ne/n:.0%})")
  ng = sum([not (graph.has_node(u) and graph.has_node(v)) for u, v in probes])
  print(f"- not in graph: {ng} ({ng/n:.0%})")
def get_distances(graph: nx.Graph, cutoff: int = None) -> (List[NodePair], List[int]):
  """
  Get all non-edges using BFS. When cutoff provided, consider only node pairs with at most this distance.
  Returns:
  - nodepairs: tuple containing all nodepairs
  - distances: tuple containing all distances
  """
  return zip(
    *[
      ((u, v), distance)
      for u, (nbs_u, _) in tqdm(nx.all_pairs_dijkstra(graph, cutoff, weight=None), total=len(graph), desc="get_distances")
      for v, distance in nbs_u.items() if distance > 1 and (cutoff is None or distance <= cutoff) 
    ]
  )

In [3]:
edges = read_edges()

In [4]:
print(f'Number of edges: {len(edges):.1e}')

Number of edges: 1.1e+06


In [9]:
edges_train_mature = filter_edgelist(edges, stop=50000)

1980-01-01 01:00:00
2000-04-17 12:01:00
no_selected_edges=50008, (4.4e-02)


In [10]:
edges_train_probe = filter_edgelist(edges, start=50000, stop=60000)

2000-04-17 12:01:01
2000-05-08 11:12:00
no_selected_edges=10008, (8.7e-03)


In [13]:
edges_test_mature = filter_edgelist(edges, stop=60000)

1980-01-01 01:00:00
2000-05-08 11:12:00
no_selected_edges=60016, (5.2e-02)


In [14]:
edges_test_probe = filter_edgelist(edges, start=60000, stop=70000)

2000-05-08 11:12:01
2000-05-23 23:10:00
no_selected_edges=10025, (8.7e-03)


## Set-up
Choose here the parameters on how you want to define the learn and assessing phase.

In [21]:
g_train_matured = giant_component(get_graph(edges_train_mature))
uv_train_probe = convert_to_set(edges_train_probe)

In [16]:
joblib.dump(g_train_matured, 'temporal/train/graph.pkl')
joblib.dump(uv_train_probe, 'temporal/train/probes.pkl')

['temporal/train/probes.pkl']

In [17]:
report(graph=g_train_matured, probes=uv_train_probe)

Number of probes: 5962
- already edge: 2554 (43%)
- both nodes in graph: 899 (15%)
- not in graph: 1690 (28%)


In [18]:
g_test_matured = giant_component(get_graph(edges_test_mature))
uv_test_probe = convert_to_set(edges_test_probe)

In [19]:
report(graph=g_test_matured, probes=uv_test_probe)

Number of probes: 6861
- already edge: 2910 (42%)
- both nodes in graph: 1118 (16%)
- not in graph: 1788 (26%)


In [20]:
joblib.dump(g_test_matured, 'temporal/test/graph.pkl')
joblib.dump(uv_test_probe, 'temporal/test/probes.pkl')

['temporal/test/probes.pkl']

## Export

### Train

In [22]:
nodepairs_train, _ = get_distances(g_train_matured, cutoff=2)
targets_train = [nodepair in uv_train_probe for nodepair in tqdm(nodepairs_train)]

100%|██████████| 2298776/2298776 [00:00<00:00, 3604406.35it/s]


In [23]:
%%time
joblib.dump(nodepairs_train, 'temporal/train/2/nodepairs.pkl', protocol=5)
joblib.dump(targets_train, 'temporal/train/2/targets.pkl', protocol=5)
joblib.dump(g_train_matured, 'temporal/train/2/graph.pkl', protocol=5)

CPU times: user 25.3 s, sys: 308 ms, total: 25.6 s
Wall time: 25.6 s


['temporal/train/2/graph.pkl']

In [27]:
print(f'{sum(targets_train) / len(nodepairs_train):e}')

5.498578e-04


### Test

In [24]:
%%time
nodepairs_test, _ = get_distances(g_test_matured, cutoff=2)
targets_test = [nodepair in uv_test_probe for nodepair in tqdm(nodepairs_test)]

100%|██████████| 2536352/2536352 [00:00<00:00, 3525554.86it/s]

CPU times: user 20.2 s, sys: 241 ms, total: 20.5 s
Wall time: 20.4 s


In [25]:
%%time
joblib.dump(nodepairs_test, 'temporal/test/2/nodepairs.pkl', protocol=5)
joblib.dump(targets_test, 'temporal/test/2/targets.pkl', protocol=5)
joblib.dump(g_test_matured, 'temporal/test/2/graph.pkl', protocol=5)

CPU times: user 27.5 s, sys: 279 ms, total: 27.8 s
Wall time: 27.8 s


['temporal/test/2/graph.pkl']

In [26]:
print(f'{sum(targets_test) / len(nodepairs_test):e}')

6.994297e-04


## Hyperparameter selection

### XGBoost

$n=2$

In [49]:
def get_x_y(df: pd.DataFrame): return df.drop(columns='target').values, df['target'].values
def gridsearch(df: pd.DataFrame, random_state=1, also_random=True, max_depth=[1, 2]) -> pd.DataFrame:
  X, y = get_x_y(df)
  
  
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=1/3, random_state=random_state)
  clf = XGBClassifier(random_state=random_state, tree_method='hist', n_jobs=6)
  gridsearch = GridSearchCV(
    clf, 
    param_grid=dict(max_depth=max_depth, scale_pos_weight=[sum(~y_train)/sum(y_train), 1]), 
    scoring='average_precision', 
    n_jobs=30,
    cv=StratifiedKFold(shuffle=True, random_state=random_state),
    return_train_score=True
  )
  
  if also_random: 
    gridsearch_random = copy.deepcopy(gridsearch)
    np.random.seed(random_state)
    y_random = copy.deepcopy(y_train)
    np.random.shuffle(y_random)
  
  gridsearch.fit(X_train, y_train)
  df_dict = dict(
      mean_train=gridsearch.cv_results_['mean_train_score'],
      std_train=gridsearch.cv_results_['std_train_score'],
      mean_val=gridsearch.cv_results_['mean_test_score'],
      std_val=gridsearch.cv_results_['std_test_score'],
      val_fold0=gridsearch.cv_results_[f'split0_test_score'],
      val_fold1=gridsearch.cv_results_[f'split1_test_score'],
      val_fold2=gridsearch.cv_results_[f'split2_test_score'],
      val_fold3=gridsearch.cv_results_[f'split3_test_score'],
      val_fold4=gridsearch.cv_results_[f'split4_test_score']
  )
  
  if also_random: 
    gridsearch_random.fit(X_train, y_random)
    df_dict['mean_train_random']=gridsearch_random.cv_results_['mean_train_score']
    df_dict['std_train_random']=gridsearch_random.cv_results_['std_train_score']
    df_dict['mean_val_random']=gridsearch_random.cv_results_['mean_test_score']
    df_dict['std_val_random']=gridsearch_random.cv_results_['std_test_score']
  df = pd.DataFrame(df_dict, index=pd.Index([(d['max_depth'], d['scale_pos_weight'] > 1) for d in gridsearch.cv_results_['params']], name=('max_depth', 'balanced')))
  df['ratio_train_val'] = (df['mean_train'] - df['mean_val'])/df['mean_val']
  df['rstd_val'] = df['std_val'] / df['mean_val']
  if also_random: df['val_over_random'] = df['mean_val'] - df['mean_val_random']
  return df.sort_values('mean_val', ascending=False)
    
def report_performance(df_train: pd.DataFrame, df_test: pd.DataFrame, random_state=1, max_depth=1, tree_method='hist', balanced=True, n_jobs=128):
  X, y = get_x_y(df_train)
  clf = XGBClassifier(max_depth=max_depth, n_jobs=128, tree_method=tree_method, scale_pos_weight=sum(~y)/sum(y) if balanced else 1 , random_state=random_state)
  clf.fit(X, y)
  X_test, y_test = get_x_y(df_test)
  y_pred = clf.predict_proba(X_test)[:,1]
  return average_precision_score(y_test, y_pred), roc_auc_score(y_test, y_pred)

In [50]:
hps2 = gridsearch(pd.read_pickle(f'temporal/train/2/features.pkl'))

In [51]:
hps2[['mean_val', 'ratio_train_val', 'rstd_val', 'val_over_random', 'val_fold0', 'val_fold1', 'val_fold2', 'val_fold3', 'val_fold4']]

mean_val  ratio_train_val  rstd_val  val_over_random  \
max_depth balanced                                                         
2         False     0.026599         0.516816  0.376079         0.026016   
          True      0.013436         0.220521  0.092344         0.012869   
1         False     0.010403         0.072873  0.196721         0.009853   
          True      0.007198        -0.001045  0.099791         0.006652   

                    val_fold0  val_fold1  val_fold2  val_fold3  val_fold4  
max_depth balanced                                                         
2         False      0.025339   0.044877   0.027025   0.015393   0.020360  
          True       0.013364   0.013894   0.012190   0.015534   0.012200  
1         False      0.010352   0.011283   0.013665   0.007599   0.009115  
          True       0.008064   0.007321   0.007687   0.006955   0.005966

In [52]:
report_performance(df_train=pd.read_pickle(f'temporal/train/2/features.pkl'), df_test=pd.read_pickle(f'temporal/test/2/features.pkl'), max_depth=1, balanced=False)

(0.011743071145453058, 0.8742894406283527)